In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
from Models.Resnet import ResNet18 as model

In [ ]:
new_net = model(num_classes=10)
print(vars(new_net))

In [3]:
def remove_filter_by_index(weight,sorted_idx,bias=None,mean=None,var=None,gate=False):       
        if mean is not None:
            mask_tensor = torch.tensor(mask_number,device=device)
            for idx in sorted_idx:
                weight[idx.item()] = mask_tensor
                bias[idx.item()] = mask_tensor
                mean[idx.item()] = mask_tensor 
                var[idx.item()] = mask_tensor
            weight = weight[weight != mask_number]
            bias = bias[bias != mask_number]
            mean = mean[mean != mask_number]
            var = var[var != mask_number]
            return weight,bias,mean,var
        elif gate:
            mask_tensor = torch.tensor(mask_number,device=device)
            for idx in sorted_idx:
                weight[idx.item()] = mask_tensor            
            weight = weight[weight != mask_number]
            return weight
        else:
            mask_tensor = torch.tensor(mask_number,device=device)
            mask_tensor = mask_tensor.repeat(list(weight[0].size()))
            for idx in sorted_idx:
                weight[idx.item()] = mask_tensor
            nonMaskRows_weight = abs(torch.abs(weight).sum(dim=(1,2,3)) - torch.abs(mask_tensor).sum(dim=(0,1,2))) > mask_number
            weight = weight[nonMaskRows_weight]
            return weight
def remove_kernel_by_index(weight,sorted_idx,linear=None):
    mask_tensor = torch.tensor(mask_number,device=device)
    mask_tensor = mask_tensor.repeat(list(weight[0][0].size()))
    for idx in sorted_idx:
        weight[:,idx.item()] = mask_tensor
    if (len(sorted_idx) != 0 and linear == None):
        nonMaskRows_weight = abs(torch.abs(weight).sum(dim=(2,3)) - torch.abs(mask_tensor).sum(dim=(0,1))) > 0.0001 
        weight = weight[:,nonMaskRows_weight[0]]
    if (linear != None):
        weight = weight[:,weight[1]!=mask_tensor]
    return weight

In [ ]:
def prune():
    new_net.conv1.weight.data = remove_filter_by_index(new_net.conv1.weight.data.clone,sorted_idx)
    new_net.layer1[0].weight.data = remove_filter_by_index(new_net.conv1.weight.data.clone,sorted_idx)